In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fifa-21-messy-raw-dataset-for-cleaning-exploring/fifa21 raw data v2.csv
/kaggle/input/fifa-21-messy-raw-dataset-for-cleaning-exploring/fifa21_raw_data.csv


In [3]:
df = pd.read_csv('/kaggle/input/fifa-21-messy-raw-dataset-for-cleaning-exploring/fifa21_raw_data.csv')
print(df.head())

                                           photoUrl  \
0  https://cdn.sofifa.com/players/158/023/21_60.png   
1  https://cdn.sofifa.com/players/020/801/21_60.png   
2  https://cdn.sofifa.com/players/200/389/21_60.png   
3  https://cdn.sofifa.com/players/192/985/21_60.png   
4  https://cdn.sofifa.com/players/190/871/21_60.png   

                       LongName  \
0                  Lionel Messi   
1  C. Ronaldo dos Santos Aveiro   
2                     Jan Oblak   
3               Kevin De Bruyne   
4    Neymar da Silva Santos Jr.   

                                           playerUrl Nationality Positions  \
0  http://sofifa.com/player/158023/lionel-messi/2...   Argentina  RW ST CF   
1  http://sofifa.com/player/20801/c-ronaldo-dos-s...    Portugal     ST LW   
2  http://sofifa.com/player/200389/jan-oblak/210005/    Slovenia        GK   
3  http://sofifa.com/player/192985/kevin-de-bruyn...     Belgium    CAM CM   
4  http://sofifa.com/player/190871/neymar-da-silv...      Brazil    

/tmp/ipykernel_33/2521719760.py:1: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/fifa-21-messy-raw-dataset-for-cleaning-exploring/fifa21_raw_data.csv')


# Ensuring the weight and height have appropriate data types

In [4]:
print(df[['Weight', 'Height']].head())

def convert_weight(weight):
    if isinstance(weight, str):
        lbs = re.findall(r'\d+', weight)
        if lbs:
            return int(lbs[0])
    return none
def convert_height(height):
    if isinstance(height, str):
        match = re.match(r"(\d+)'(\d+)", height)
        if match:
            feet = int(match.group(1))
            inches = int(match.group(2))
            return feet * 12 + inches  # Convert height to total inches
    return None

# Apply the conversion functions
df['weight_lbs'] = df['Weight'].apply(convert_weight)
df['height_in'] = df['Height'].apply(convert_height)

# Display the first few rows to check the results
print(df[['Weight', 'weight_lbs', 'Height', 'height_in']].head())

   Weight Height
0  159lbs   5'7"
1  183lbs   6'2"
2  192lbs   6'2"
3  154lbs  5'11"
4  150lbs   5'9"
   Weight  weight_lbs Height  height_in
0  159lbs         159   5'7"         67
1  183lbs         183   6'2"         74
2  192lbs         192   6'2"         74
3  154lbs         154  5'11"         71
4  150lbs         150   5'9"         69


# Separating joined column into year, month, day columns

In [5]:
from datetime import datetime
print(df['Joined'].head())
def split_joined_date(joined):
    if isinstance(joined, str):
        try:
            date= datetime.strptime(joined, '%b %d, %Y')
            return date.year, date.month, date.day
        except ValueError:
            return None, None, None
    return None, None, None
df['Year'],df['Month'], df['Day'] = zip(*df['Joined'].apply(split_joined_date))

print(df[['Joined', 'Year', 'Month','Day']].head())

0     Jul 1, 2004
1    Jul 10, 2018
2    Jul 16, 2014
3    Aug 30, 2015
4     Aug 3, 2017
Name: Joined, dtype: object
         Joined  Year  Month  Day
0   Jul 1, 2004  2004      7    1
1  Jul 10, 2018  2018      7   10
2  Jul 16, 2014  2014      7   16
3  Aug 30, 2015  2015      8   30
4   Aug 3, 2017  2017      8    3


# Cleaning and trasnforming the value, wage,and release cause columns into column of integers

In [6]:
def clean_and_convert(value):
    if isinstance(value, str):
        # Remove the euro sign and replace K and M with e3 and e6 respectively
        value = value.replace('€', '').replace('K', 'e3').replace('M', 'e6')
        try:
            # Evaluate the string to convert it to an integer
            return int(float(value))
        except ValueError:
            return None
    return None
print(df['Value'].dtype)
print(df['Value'])

object
0        €67.5M
1          €46M
2          €75M
3          €87M
4          €90M
          ...  
18974      €35K
18975      €60K
18976      €40K
18977      €60K
18978      €60K
Name: Value, Length: 18979, dtype: object


In [7]:
df['Value'] = df['Value'].apply(clean_and_convert)
df['Wage'] = df['Wage'].apply(clean_and_convert)
df['Release Clause'] = df['Release Clause'].apply(clean_and_convert)
print(df[['Value', 'Wage', 'Release Clause']].tail())

       Value  Wage  Release Clause
18974  35000  1000           57000
18975  60000   500          165000
18976  40000  1000           70000
18977  60000   500          165000
18978  60000   500          167000


# Removing the newlline characters from the Hits column

In [9]:
df['Hits'] = df['Hits'].str.replace('\n','',regex=False)
# Print the results
print(df[['Value', 'Wage', 'Release Clause', 'Hits']].head())

      Value    Wage  Release Clause Hits
0  67500000  560000       138400000  372
1  46000000  220000        75900000  344
2  75000000  125000       159400000   86
3  87000000  370000       161000000  163
4  90000000  270000       166500000  273


# Separating the Team and Contract column into seperate team and contract columns

In [ ]:
df[['Team', 'Contract']] = df['Team & Contract'].str.split(' ', 1, expand=True)